In [0]:
!pip install tensorflow==1.15

  Using cached https://files.pythonhosted.org/packages/3f/98/5a99af92fb911d7a88a0005ad55005f35b4c1ba8d75fba02df726cd936e6/tensorflow-1.15.0-cp36-cp36m-manylinux2010_x86_64.whl


In [0]:
#Libraries
import pandas as pd
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
from spacy.lang.pt.examples import sentences
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.decomposition import PCA
import pickle
#nlp = spacy.load('pt')
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE 
import matplotlib.pyplot as plt
#pip install scipy==0.10.1
import gensim
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from __future__ import division, print_function
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.preprocessing import text

Using TensorFlow backend.


In [0]:
import keras
keras.__version__

'2.3.1'

In [0]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [0]:
#Take Files From GoogleDrive
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf

In [0]:
import os
os.chdir('/content/drive/My Drive/Research/DataSets/')
path= '/content/drive/My Drive/Research/DataSets/'
# file = path+'glove_s1000.zip'
# !unzip glove_s1000.zip

In [0]:
embeddings_index = {}
word_list=[]
f = open('glove_s300.txt', errors = 'ignore', encoding='utf-8')
print(f)
for line in f:
  # values = line.split()
  # word = ''.join(values[0])
  # word_list.append(word)
  # coefs = np.asarray(values[1:], dtype='float32')
  # embeddings_index[word] = coefs
  values = line.split()
  word = ''.join(values[:-300])
  word_list.append(word)
  coefs = np.asarray(values[-300:], dtype='float32')
  embeddings_index[word] = coefs
f.close()


<_io.TextIOWrapper name='glove_s300.txt' mode='r' encoding='utf-8'>


In [0]:
#MAX_SEQUENCE_LENGTH = len(train_word_index)+1
MAX_SEQUENCE_LENGTH = 5000
EMBEDDING_DIM = 300

In [0]:
%run '/content/drive/My Drive/Fraud detection in internal auditing/Colab Notebooks/NLP functions_Gus.ipynb'
# %run '/content/drive/My Drive/Colab Notebooks/NLP-4-Classification-FUNCTIONS.ipynb'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'

In [0]:
def overSamplingRISKs(df,rate):
  X = df 
  not_HIGH = df[df.RISK!="HIGH"]
  HIGH = df[df.RISK=="HIGH"]
  oversample_by= int((rate*((len(df[df.RISK!="HIGH"])+len(df[df.RISK=="HIGH"]))-(len(df[df.RISK=="HIGH"]))))/(1-rate))
  # OVERsample minority
  high_upsampled = resample(HIGH,
                            replace=True, # sample with replacement
                            n_samples=oversample_by, # match number in majority class
                            random_state=27) # reproducible results

  upsampled = pd.concat([not_HIGH, high_upsampled])

  print( upsampled.RISK.value_counts() )

  return upsampled

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
from keras.layers import MaxPooling1D

#list_of_years=['2016_DF_with_cleaned_stemmed_nonstemmed_v01.02.20.pkl','2017_DF_with_cleaned_stemmed_nonstemmed_v01.02.20.pkl','2018_DF_with_cleaned_stemmed_nonstemmed_v01.02.20.pkl','2019_DF_with_cleaned_stemmed_nonstemmed_v01.02.20.pkl']
list_of_years=['2016_DF_cleaned_not_balanced.pkl']
label_names = ['Risk_HIGH', 'Risk_MEDIUM-LOW']
for ele in list_of_years:
  filename = DATASETS_FOLDER+ele
  df = pd.read_pickle(filename)
  datatrain,datatest= train_test_split(df, test_size=0.30, random_state=42)
  datatrain = overSamplingRISKs(datatrain,0.30)
  Training_vocabulary = sorted(list(set(word for tokens in datatrain.token_filtered for word in tokens)))
  #tokenizer= text.Tokenizer(num_words=5000,lower=True, char_level=False)
  tokenizer= text.Tokenizer(num_words=5000)
  tokenizer.fit_on_texts(datatrain.sent_token_final.tolist())
  training_sequences = tokenizer.texts_to_sequences(datatrain.sent_token_final.tolist())
  train_word_index = tokenizer.word_index
  train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
  #train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
  train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
  for word, index in train_word_index.items():
    try:
      embedding_vector = embeddings_index[word]
      train_embedding_weights[index] = embedding_vector
    except KeyError:
      train_embedding_weights[index] = np.zeros(EMBEDDING_DIM)

  label_names = ['Risk_HIGH', 'Risk_MEDIUM-LOW']
  model = Sequential()
  model.add(Embedding(len(train_word_index)+1, EMBEDDING_DIM, weights=[train_embedding_weights], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
  model.add(Dropout(0.5))
  convs = []
  filter_sizes = [2,3,4,5,6]
  for filter_size in filter_sizes:
    model.add(Conv1D(filters=200, kernel_size=filter_size, activation='relu',padding='valid'))
    model.add(MaxPooling1D())
  #   l_pool = GlobalMaxPooling1D()(l_conv)
  #   convs.append(l_pool)
  # l_merge = concatenate(convs, axis=1)      ,return_sequence=False
  model.add(Dropout(0.1))
  model.add(Dense(256,activation='relu'))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(len(list(label_names)),activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

  test_sequences = tokenizer.texts_to_sequences(datatest.sent_token_final.tolist())
  test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

  x_train = train_cnn_data
  y_train = datatrain[label_names].values
  y_tr = y_train

  x_test= test_cnn_data
  y_test= datatest[label_names].values
  y_te = y_test
  
  model.fit(x_train,y_tr, batch_size=32, epochs=20,validation_data=(x_test,y_te))
  predictions_TRAIN=model.predict(train_cnn_data, batch_size=32, verbose=1)
  predictions= model.predict(test_cnn_data, batch_size=32, verbose=1)


MEDIUM-LOW    12467
HIGH           5343
Name: RISK, dtype: int64
Train on 17810 samples, validate on 5531 samples
Epoch 1/20
17810/17810 [==============================] - 140s 8ms/step - loss: 0.5371 - accuracy: 0.7214 - val_loss: 0.3781 - val_accuracy: 0.8228
Epoch 2/20
17810/17810 [==============================] - 138s 8ms/step - loss: 0.3532 - accuracy: 0.8316 - val_loss: 0.3140 - val_accuracy: 0.8695
Epoch 3/20
17810/17810 [==============================] - 138s 8ms/step - loss: 0.2137 - accuracy: 0.9047 - val_loss: 0.3479 - val_accuracy: 0.8849
Epoch 4/20
17810/17810 [==============================] - 138s 8ms/step - loss: 0.1559 - accuracy: 0.9290 - val_loss: 0.3159 - val_accuracy: 0.8951
Epoch 5/20
17810/17810 [==============================] - 137s 8ms/step - loss: 0.1256 - accuracy: 0.9386 - val_loss: 0.3947 - val_accuracy: 0.8789
Epoch 6/20
17810/17810 [==============================] - 137s 8ms/step - loss: 0.1075 - accuracy: 0.9476 - val_loss: 0.4083 - val_accuracy: 0.886

In [0]:
predictions_TRAIN1= np.asarray([np.argmax(x) for x in predictions_TRAIN])
predictions_1= np.asarray([np.argmax(x) for x in predictions])
  

In [0]:
acc, pre, rec, f1s =get_metrics(datatrain.FinalLabel,predictions_TRAIN1)
acc, pre, rec, f1s =get_metrics(datatest.FinalLabel,predictions_1)

Accuracy: 0.9617  Precision: 1.0  Recall: 0.9452  F1 Score: 0.9718
Accuracy: 0.891  Precision: 0.9745  Recall: 0.9108  F1 Score: 0.9416
